# Amuse tutorial

To start with AMUSE, first open an iPython terminal (or Jupyter notebook).

To use AMUSE, it must first be imported.
A quick and dirty way to do this (preferably not used in production, but it can be useful for quick testing) is:
```python
from amuse.lab import *
```
In production, it is better to just import the specific modules needed, to improve the clarity of the code:

In [ ]:
from amuse.datamodel import Particles
from amuse.units import units, constants
from amuse.ic.plummer import new_plummer_model

## Quantities and units

One of the features of AMUSE is that it has Quantities, e.g. values with a unit attached to it.
Units can be automatically converted to other units of the same type (e.g. parsec to lightyear), while an operation which should be illegal (e.g. adding a length to a mass) will result in an incompatible units exception:

In [ ]:
distance = 1 | units.parsec
another_distance = 1 | units.lightyear
print(distance + another_distance)
print((distance + another_distance).in_(units.AU))
weight = 1 | units.MSun
distance + weight

We can also define new units:

In [ ]:
from amuse.units.derivedsi import named

lightminute = named(
    "lightminute",  # Name of the unit
    "lmin",  # Symbol
    (constants.c * units.minute).to_unit()  # the unit to be named
)
print(1 | lightminute)
print((1 | lightminute).in_(units.AU))

A VectorQuantity is the equivalent of a Numpy array with a single unit:

In [ ]:
position = [1.0, 0.0, 0.0] | units.AU
print(position)
print(1 | lightminute)
print((1 | lightminute).in_(units.lightyear))
print(position.in_(lightminute))

Amuse also supports non-SI units, e.g. "generic" units (as may be used internally in codes) and "N-body" or Hénon units (a specifically normalised system in which G=1, often used in N-body codes):

In [ ]:
from amuse.units import generic_unit_system, nbody_system

To convert between internal units of a code and physical units, we use a converter. The converter determines the scale of a problem in a code, so it is important to choose the right scale units.

In [ ]:
mass_scale = 100 | units.MSun
length_scale = 3 | units.parsec
converter = nbody_system.nbody_to_si(mass_scale, length_scale,)
print(converter.to_nbody(100 | units.MSun))
print(converter.to_nbody(1 | units.parsec))
print(converter.to_nbody(1 | units.Myr))
print(converter.to_si(1 | nbody_system.time).in_(units.Myr))
print(converter.to_si(1 | nbody_system.speed).in_(units.km * units.s**-1))

At the core of AMUSE is the particles datamodel. A particle can have any number of attributes, a particleset is a set of particles with the same attributes defined. 
Each particle has a unique key, which is randomly generated and never repeated.

In [ ]:
from amuse.datamodel import Particles, Particle

p0 = Particle()
p0.name = "Particle Zero"
p = Particles()

print(p0)
print(p)

Particles can be added to or subtracted from particlesets (note that the same particle can be in a particleset more than once):

In [ ]:
p.add_particle(p0)
q = Particles(3)
q.name = list("Particle %i" % i for i in range(1,1+len(q)))
print(q)
p.add_particles(q)
print(p)
p.remove_particle(q[0])
print(p)

Particlesets support many useful operations, like center_of_mass; center_of_mass_velocity; kinetic_energy etc.

In [ ]:
speed = named("speed", "speed", nbody_system.length * nbody_system.time**-1)
energy = named("energy", "energy", nbody_system.length**2 * nbody_system.time**-2 * nbody_system.mass)

from amuse.ic.plummer import new_plummer_model
particles = new_plummer_model(100)
print(particles.center_of_mass())
print(particles.center_of_mass_velocity().in_(speed))
print(particles.kinetic_energy().in_(energy))
print(particles.potential_energy(G=nbody_system.G).in_(energy))

A lot of "standard" initial condition generators are defined in `amuse.ic`, e.g. for generating a Plummer sphere, a Salpeter initial mass function, and many more (have a look!). Some other, more complex initial condition generators can be found in `amuse.ext`, e.g. for a molecular cloud.

In [ ]:
from amuse.ext.molecular_cloud import molecular_cloud

converter = nbody_system.nbody_to_si(5000 | units.MSun, 5 | units.parsec)

mc = molecular_cloud(targetN=10000, convert_nbody=converter).result

In [ ]:
print("Centre of mass: ", mc.center_of_mass().in_(units.parsec))
print("Centre of mass velocity: ", mc.center_of_mass_velocity().in_(units.kms))
print("Mean distance to the coordinate centre: ", mc.position.lengths().mean().in_(units.parsec))